In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##### Importing Libraries

In [ ]:
!pip install datasets transformers[torch] accelerate -U evaluate

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset,DatasetDict
import evaluate
import numpy as np
import torch
import gc

##### Loading our dataset

In [ ]:
# Load dataset
dataset = load_dataset("yelp_review_full")

##### Rows selection for faster computation

In [ ]:
# Number of rows to select
num_rows_train = 10000
num_rows_test = 5000

# Shuffling and selecting 5000 rows from the training dataset
train_ds_small = dataset['train'].shuffle(seed=42).select(range(num_rows_train))

# Shuffling and selecting 5000 rows from the testing dataset
eval_ds_small = dataset['test'].shuffle(seed=42).select(range(num_rows_test))

small_dataset = DatasetDict({
    'train': train_ds_small,
    'validation': eval_ds_small
})

##### Tokenization

In [ ]:
# Define tokenizer
tokenizer = AutoTokenizer.from_pretrained("LiYuan/amazon-review-sentiment-analysis")

def tokenize_function(examples):
   # Map function
    # padding and truncation control for variable length sequences
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# apply to all datasets with .map(). Built in function of the HF datasets class
tokenized_datasets = small_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

##### Metric Computation Function

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": accuracy, "f1": f1}

##### Set of random parameters for optimization

In [ ]:
# Define sets of hyperparameters
param_sets = [
    {
        "learning_rate": 2e-5,
        "num_train_epochs": 2,
        "per_device_batch_size": 16,
        "weight_decay": 0.001,
        "warmup_steps":200
    },
    {
        "learning_rate": 2e-5,
        "num_train_epochs": 3,
        "per_device_batch_size": 16,
        "weight_decay": 0.001,
        "warmup_steps":300
    },
    {
        "learning_rate": 3e-5,
        "num_train_epochs": 2,
        "per_device_batch_size": 16,
        "weight_decay": 0.001,
        "warmup_steps":300
    },
    {
        "learning_rate": 4e-5,
        "num_train_epochs": 3,
        "per_device_batch_size": 32,
        "weight_decay": 0.001,
        "warmup_steps":400
    },
    {
        "learning_rate": 5e-5,
        "num_train_epochs": 3,
        "per_device_batch_size": 32,
        "weight_decay": 0.001,
        "warmup_steps":500
    }
]

## Training with best parameters and pushing to Hugging Face

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
repo_name = "yelp-review-sentiment-analysis"

In [ ]:
best_model = None
best_accuracy = 0
best_params = None

for idx, params in enumerate(param_sets):
    # Clear memory
    torch.cuda.empty_cache()
    gc.collect()

    training_args = TrainingArguments(
        output_dir=f"yelp-review-sentiment-analysis-model-{idx+1}",
        learning_rate=params["learning_rate"],
        per_device_train_batch_size=params["per_device_batch_size"],
        per_device_eval_batch_size=params["per_device_batch_size"],
        num_train_epochs=params["num_train_epochs"],
        weight_decay=params["weight_decay"],
        warmup_steps=params["warmup_steps"],
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=False,  # Do not push during training
        fp16=True  # Enable mixed precision training
    )

    model = AutoModelForSequenceClassification.from_pretrained("LiYuan/amazon-review-sentiment-analysis")

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_results = trainer.evaluate()
    accuracy = eval_results['eval_accuracy']
    f1 = eval_results['eval_f1']

    print(f"Params: {params}")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = trainer
        best_params = params
        best_tokenizer = tokenizer

if best_model:
    print(f"Best Params: {best_params}")
    print(f"Best Accuracy: {best_accuracy}")

    # Push the best model to the Hub
    best_model.push_to_hub()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.929200,0.879833,0.614200,0.618963
2,0.754300,0.876481,0.631600,0.629445


Params: {'learning_rate': 2e-05, 'num_train_epochs': 2, 'per_device_batch_size': 16, 'weight_decay': 0.001, 'warmup_steps': 200}
Accuracy: 0.6316
F1 Score: 0.6294448903674568


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.935200,0.894287,0.609600,0.614130
2,0.780500,0.893029,0.627800,0.622806
3,0.638100,0.965274,0.629200,0.629810


Params: {'learning_rate': 2e-05, 'num_train_epochs': 3, 'per_device_batch_size': 16, 'weight_decay': 0.001, 'warmup_steps': 300}
Accuracy: 0.6278
F1 Score: 0.6228056008491232


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.949000,0.919883,0.598600,0.603465
2,0.765400,0.882166,0.630800,0.629222


...

events.out.tfevents.1720238693.ca79502f6b92.732.0:   0%|          | 0.00/6.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

events.out.tfevents.1720239393.ca79502f6b92.732.1:   0%|          | 0.00/457 [00:00<?, ?B/s]

TypeError: PushToHubMixin.push_to_hub() missing 1 required positional argument: 'repo_id'

##### Testing the optimized model for example dataset

In [ ]:
from transformers import pipeline

In [ ]:
#loading our model
my_model = pipeline(model="harmanpreet-kaur/yelp-review-sentiment-analysis-model-1")

In [23]:
#example data
data = [
    "The worst dental office I ever been. No one can beat it!!! You should avoid it at any time.",
    "Some of the best chow around--love this place. The bread and salads and soups are great.",
    "I love this place! The food is always so fresh and delicious. The staff is always friendly, as well.",
    "Papa J is expensive and inconsistent. The ambiance is nice but it does not justify the prices. There are better restaurants in Carnegie."
]

In [24]:
#printing the results
my_model(data)

[{'label': '1 star', 'score': 0.9909992218017578},
 {'label': '5 stars', 'score': 0.7539454102516174},
 {'label': '5 stars', 'score': 0.9274201393127441},
 {'label': '2 stars', 'score': 0.6682943105697632}]